### Initial Imports

In [1]:
import sys
sys.path.insert(0, 'C:/Users/enioh/Documents/Github/MMA-ML-Model')

import pandas as pd 
import numpy as np 

from os.path import abspath, dirname, join

### Dataset and Model Loading

In [2]:
df = pd.read_csv(abspath(join(dirname(abspath('')), 'data/', 'processed/', 'extracted_stats.csv')))

In [4]:
from src.models.XGBoost_Model import make_model
from src.models.train_model import train_model
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

clf = make_model()
clf, split_data = train_model(clf, df)

X_train, y_train, X_test, y_test = split_data

C:\Users/enioh/Documents/Github/MMA-ML-Model\src\models\train_model.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  post_comp_cols = df.loc[:, ~df.columns.str.contains('precomp_([a-zA-Z_]+)_vs_opp', regex=True)].columns.to_list() # All columns that do not have the precomp_ id or vs_opp id


Model Training Accuracy: 0.727378981201909
Model Test Accuracy: 0.5831710167510713


### Thresholding Selection

In [9]:
def threshold_selector(clf, X_train, y_train, X_test, y_test):

    max_features = [x for x in range(100, 300, 1)]
    avgs = []

    for f in max_features:
        loopavg = []
        for count in range(0,1):
            selection = SelectFromModel(estimator=clf, max_features=f, prefit=True)
            select_X_train = selection.transform(X_train)

            model = make_model()
            model.fit(select_X_train, y_train)
            select_X_test = selection.transform(X_test)
            y_pred = model.predict(select_X_test)
            accuracy = accuracy_score(y_test, y_pred)

            msg = f"Thresh={f}, n={select_X_train.shape[1]}, Accuracy: {accuracy}"
            print(msg)
            loopavg.append(accuracy)
        avgs.append((f, np.mean(loopavg)))
    
    for avg in avgs:
        print(avg)

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
threshold_selector(clf, X_train, y_train, X_test, y_test)

### Recursive Feature Elimination

In [41]:
from sklearn.feature_selection import RFE

clf = make_model()
selector = RFE(clf, n_features_to_select=25, step=5)
selector.fit(X_train, y_train)

RFE(estimator=XGBClassifier(alpha=1, base_score=None, booster=None,
                            callbacks=None, colsample_bylevel=None,
                            colsample_bynode=None, colsample_bytree=0.1,
                            early_stopping_rounds=None,
                            enable_categorical=False, eval_metric=None,
                            feature_types=None, gamma=0.05, gpu_id=None,
                            grow_policy=None, importance_type=None,
                            interaction_constraints=None, learning_rate=0.02,
                            max_bin=None, max_cat_threshold=None,
                            max_cat_to_onehot=None, max_delta_step=0,
                            max_depth=3, max_leaves=None, min_child_weight=16,
                            missing=nan, monotone_constraints=None,
                            n_estimators=1000, n_jobs=-1,
                            num_parallel_tree=None, predictor=None, ...),
    n_features_to_select=25, step=5)